<a href="https://colab.research.google.com/github/Kadomium/Car_kit/blob/main/Car_kit_prediction_null_drop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell, Dropout

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%pip install japanize-matplotlib
import japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=05aeff5bccba230cbedf754f6362db93143b12bbc9871661c7211d043824c24b
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize-matplotlib


In [ ]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [ ]:
# --- Step 1: ファイルパスと設定の定義 ---
new_file_path = 'carkit_null_drop.csv'
chunk_size = 50000 # PCのスペックに応じて調整

# 列名をリストとして取得
new_all_columns = pd.read_csv(new_file_path, nrows=0).columns.tolist()

# ---「削除する列」のリストを完成させる ---
first_columns_to_drop = (['key','qqkey','qxq999','qs3','qsk4_1c','qsk4_2c','qs4_3','qs4_8a','qs4_9','qs4_10','qs4_11','qs4_12','qs4_13','qsq4_gas','qs4_4a','qskei_1','qskei_2','qsnou_1','qsnou_2','q6_2','q6_4']+[f'q6_5_3{x}{i}' for x in range(3, 6) for i in range(1, 22)]+['q9_2','q12_2']++[f'q13_1{i}' for i in range(1, 29)]+['q13_2','q14_2','qsq22_1'+'q29_2','q34_2','q42_4','qw1','qw2','qw11','qseinen','_src'])

#q22に連続性の問題がなさそうなことから使用したい、バイナリデータも作成済み
extra_columns_to_drop = ['qsk4_15','qs4_4a']

# --- Step 5: 【作戦実行】「保持する列」リストを作成し、本番の処理へ ---
# バイナル作成前に「保持すべき列」のリストを作成
first_columns_to_keep = [col for col in new_all_columns if col not in first_columns_to_drop]
columns_to_keep = [col for col in first_columns_to_keep if col not in extra_columns_to_drop]
print(f"\n全1646列のうち、バイナル作成前に保持する列は {len(first_columns_to_keep)} 個です。")
print(f"\n全1646列のうち、バイナル作成後に保持する列は {len(columns_to_keep)} 個です。")

発見！前回データには存在しなかった新しい列が 25 個あります:
Nullが1つ以上存在する列が 879 個見つかりました。


null1個以上:1126

In [ ]:
#if 'qsk4_15'=428のバイナリを作成し、yとする（その後qsk4_15をdrop）
#連続性をq22を中心に認めるなら、正直'_src'のdropの前にやることってなくない？

In [ ]:
#ここも、もし乗っかればデータ型の最適化以外使わなくない？

# --- Step 3: チャンクごとにデータを処理するループ ---
# 処理済みのチャンクを格納するための空のリスト
processed_chunks = []

# イテレータ（繰り返し処理の仕組み）を作成
# usecolsで、必要な列だけを読み込むように指定！
chunk_iterator = pd.read_csv(new_file_path, chunksize=chunk_size, usecols=first_columns_to_keep)

#print("--- 巨大ファイルのチャンク処理を開始 ---")
for i, chunk in enumerate(chunk_iterator):
    #print(f"チャンク {i+1} を処理中 (行数: {len(chunk)})...")

    # --- ここで、各チャンクに対して必要な前処理を行う ---

    # (例) データ型の最適化 (メモリ削減に超重要！)
    for col in chunk.select_dtypes(include=['int64']).columns:
        chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
    for col in chunk.select_dtypes(include=['float64']).columns:
        chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    for col in chunk.select_dtypes(include=['object']).columns:
        if chunk[col].nunique() / len(chunk) < 0.5:
            chunk[col] = chunk[col].astype('category')

    # (例) Nullの処理（もし必要なら）
    # chunk.dropna(subset=['important_column'], inplace=True)

    # 処理済みのチャンクをリストに追加
    processed_chunks.append(chunk)

#print("\n--- 全てのチャンク処理が完了 ---")


# --- Step 4: 処理済みの全チャンクを一つのデータフレームに結合 ---
#print("処理済みチャンクを結合中...")
df_cleaned = pd.concat(processed_chunks, ignore_index=True)
##print("結合完了！")

#print("\n--- 最終的なデータフレーム情報 ---")
df_cleaned.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77512 entries, 0 to 77511
Columns: 1321 entries, qstrend to _src
dtypes: category(1), float32(667), int16(1), int8(652)
memory usage: 245.6 MB


In [ ]:
df_cleaned.loc[298143, '_src'] = '20250831_Car-kit_2023-2025_raw.dta'

In [ ]:
df_cleaned.tail()

,qstrend,qxxx_1,qsk4_15,qs4_4a,qs7,qs7_2,qs8,qs8_2,qs9,qxxx_2,...,q57_114,q57_115,q57_116,q57_117,q57_118,qw3,qw5p,qw6,qw7,_src
298139,126,NaN,951,130,1,2,1,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,46.0,13.0,1.0,10.0,20250831_Car-kit_2023-2025_raw.dta
298140,126,NaN,335,130,1,1,2,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,8.0,4.0,1.0,3.0,20250831_Car-kit_2023-2025_raw.dta
298141,126,NaN,436,130,2,3,1,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,3.0,7.0,1.0,2.0,20250831_Car-kit_2023-2025_raw.dta
298142,126,NaN,995,130,1,1,1,1,2,NaN,...,0.0,0.0,0.0,0.0,0.0,21.0,1.0,1.0,6.0,20250831_Car-kit_2023-2025_raw.dta
298143,126,NaN,257,130,1,1,1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20250831_Car-kit_2023-2025_raw.dta


In [ ]:
#qsk4_15=428がN-BOX、430がN-BOX Custom
# qsk4_15が428なら1、そうでなければ0の列を作成

df_cleaned['target_count_custom'] = (df_cleaned['qsk4_15'] == 430).astype(int)

# ★★★★★ このタイミングで 'qsk4_15' を完全に削除する ★★★★★
df_cleaned.drop(columns=['qsk4_15'], inplace=True)
#
columns_to_keep = first_columns_to_keep

In [ ]:
# 1. 各列のnullの数を計算
nan_counts = df_cleaned.isnull().sum()
# 2. nullの数を降順でソートして表示
print("--- 各列のNullの数（降順） ---")
print(nan_counts.sort_values(ascending=False))

# 3. データ総数に対するnullの割合を計算して表示
total_rows = len(df_cleaned)
nan_ratio = (df_cleaned.isnull().sum() / total_rows) * 100
print("\n--- 各列のNullの割合（%） ---")
print(nan_ratio.sort_values(ascending=False))

In [ ]:
# 残った列で再度、欠損率を計算
nan_ratio = df.isnull().sum() / len(df)

# 可視化1: ヒストグラムで欠損率の分布を見る
plt.figure(figsize=(12, 6))
sns.histplot(nan_ratio * 100, bins=50, kde=True)
plt.title('欠損率の分布（100%欠損の列は削除済み）', fontsize=16)
plt.xlabel('欠損率 (%)', fontsize=12)
plt.ylabel('質問項目の数', fontsize=12)
plt.grid(True)
plt.savefig("null_hist.png", dpi=300, bbox_inches='tight')
plt.show()

# 可視化2: 欠損率を降順にソートした棒グラフで見る
plt.figure(figsize=(12, 6))
nan_ratio.sort_values(ascending=False).plot(kind='line', use_index=False)
plt.title('欠損率の降順プロット（エルボーポイントを探す）', fontsize=16)
plt.xlabel('質問項目（欠損率の高い順）', fontsize=12)
plt.ylabel('欠損率', fontsize=12)
plt.grid(True)
plt.savefig("null_elbow.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# --- Step 1: 時間コードと実際の年月を対応付ける辞書を作成 ---
# このマッピングを37ヶ月分作成します
start_year = 2017
start_month = 12
time_map = {}
for i in range(91):
    current_year = start_year + (start_month + i - 1) // 12
    current_month = (start_month + i - 1) % 12 + 1

    time_code = 54 + i
    date_str = f"{current_year}-{current_month:02d}" # '2022-01' のように0埋めする

    time_map[time_code] = date_str

# --- Step 2: 辞書を使って 'qs4_4a'列を置換（マッピング） ---
# .map()メソッドを使うのが最も簡単で効率的です
df_cleaned['date_str'] = df_cleaned['qs4_4a'].map(time_map)

# --- Step 3: 【最重要】文字列をPandasのdatetime型（月末実績値）に変換 ---
df_cleaned['date'] = pd.to_datetime(df_cleaned['date_str']).dt.to_period('M').dt.to_timestamp('M')

# --- Step 4: datetime型の列をデータフレームのインデックスに設定 ---
# これにより、データが正式に時系列データとして扱われます
df_cleaned.set_index('date', inplace=True)

# 元の不要になった列は削除して整理
df_cleaned.drop(['qs4_4a', 'date_str'], axis=1, inplace=True)

print("\n--- 最終的な時系列データフレーム ---")
print(df_cleaned.head())
print("\nインデックスの型:")
print(type(df_cleaned.index))

In [ ]:
# Step 1の分類に基づき、集計戦略を立てる
# まず、4.多クラスのカテゴリデータ(職業、最も運転する人など)をOne-Hotエンコーディングする
df_cleaned = pd.get_dummies(df_cleaned, columns=['qs9','q13_2a','qw3','qw7'])

# Step 2: .agg() を使って一気に集計する
#エンコーディング済みはqxx_iのように下に_（選択肢）がつく
# 集計ルールを辞書で定義
#1. 連続する数値データ(年齢)　は : 'mean', 外れ値が嫌なら : 'median',
#2. 段階的な数値データ(満足度、こだわり度) : 'mean',
#3. バイナリデータは : 'mean',

aggregation_rules = {
    col: 'sum' if col == 'target_count_custom' else 'mean'
    for col in columns_to_aggregate
}

# 'M' (Month End) でリサンプリングし、定義したルールで集計
df_monthly = df_cleaned.resample('M').agg(aggregation_rules)

print("--- 月次に集計されたデータフレーム ---")
print(df_monthly.head())

In [ ]:
# --- Step 1: まず「特徴量X」と「正解ラベルy」に分離する ---
X = df_monthly.drop('target_count_custom', axis=1) # target_count_custom列以外、すべてが特徴量X
y = df_monthly[['target_count_custom']] # target_count_custom列だけが正解ラベルy (DataFrame形式で抽出)

print("--- 1. 特徴量Xと正解ラベルyに分離 ---")
print("X (問題用紙) の形状:", X.shape)
print("y (模範解答) の形状:", y.shape)


# --- Step 2: Xとyを、同じ分割点で7:2:1に分割する ---
n_data = len(df_monthly)
train_split = int(n_data * 0.7)
val_split = int(n_data * 0.9)

X_train, X_val, X_test = X[:train_split], X[train_split:val_split], X[val_split:]
y_train, y_val, y_test = y[:train_split], y[train_split:val_split], y[val_split:]

print(f"\n--- 2. Xとyを同じように時間で分割 ---")
print(f"学習データ: X_train:{X_train.shape}, y_train:{y_train.shape}")
print(f"検証データ: X_val:{X_val.shape}, y_val:{y_val.shape}")
print(f"テストデータ: X_test:{X_test.shape}, y_test:{y_test.shape}")


# --- Step 3: Xとyを「別々の」Scalerで正規化する ---
# ★★★★★ なぜ別々？ -> 後でyの予測結果を元のスケールに戻すため ★★★★★
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

# (3-a) Scalerを「学習データだけ」にフィットさせる
scaler_X.fit(X_train)
scaler_y.fit(y_train)

# (3-b) 学習したScalerを使って、全てのデータを変形(transform)する
X_train_scaled = scaler_X.transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

print(f"\n--- 3. 正しく正規化されたXとyのデータ ---")
print("X_train_scaledの形状:", X_train_scaled.shape)
print("y_train_scaledの形状:", y_train_scaled.shape) # LSTMに入力するため、こちらも正規化

In [ ]:
# --- Step 1: ルックバック期間の定義 ---
lookback = 3 # 過去3ヶ月分のデータを使って予測する


# --- Step 2: シーケンスデータを作成する関数を定義 ---
def create_sequences(X_data, y_data, lookback):
    X_seq, y_seq = [], []
    # ループは lookback から開始し、データの最後まで
    for i in range(lookback, len(X_data)):
        # i-lookback から i までが入力シーケンス (問題)
        X_seq.append(X_data[i-lookback:i, :])
        # i 番目が正解ラベル (答え)
        y_seq.append(y_data[i, 0])

    return np.array(X_seq), np.array(y_seq)


# --- Step 3: 関数を各データセットに適用 ---
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, lookback)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, lookback)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, lookback)


# --- Step 4: 結果の形状を確認 ---
print("--- シーケンスデータへの変換後 ---")
print("【学習用データ】")
print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"  -> {X_train_seq.shape[0]}個の「問題と答えのペア」が作成されました。")

print("\n【検証用データ】")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"y_val_seq shape: {y_val_seq.shape}")
print(f"  -> {X_val_seq.shape[0]}個の「問題と答えのペア」が作成されました。")

print("\n【テスト用データ】")
print(f"X_test_seq shape: {X_test_seq.shape}")
print(f"y_test_seq shape: {y_test_seq.shape}")
print(f"  -> {X_test_seq.shape[0]}個の「問題と答えのペア」が作成されました。")

In [ ]:
# --- モデルの器を定義 ---
model = Sequential()

# --- 入力層 兼 1層目のLSTM ---
# units: LSTM層のニューロン数。多すぎず少なすぎず64あたりから始めるのが一般的。
# input_shape: モデルへの最初の入力の形状。(タイムステップ数, 特徴量数)
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2)) # 20%のニューロンをランダムに無効化

# --- 2層目のLSTM ---
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))

# --- 全結合層 ---
model.add(Dense(units=16, activation='relu')) # reluは標準的な活性化関数

# --- 出力層 ---
model.add(Dense(units=1)) # 最終的な予測値は1つなのでunits=1


# --- モデルのコンパイル ---
# optimizer: どうやって学習を進めるかのアルゴリズム。'adam'が最も一般的で高性能。
# loss: モデルの予測がどれだけ間違っているかの指標。回帰問題では'mean_squared_error'が標準。
model.compile(optimizer='adam', loss='mean_squared_error')


# --- モデルの設計図を確認 ---
model.summary()

In [ ]:
# --- EarlyStoppingの設定 ---
# patience=5: 5エポック連続で検証ロスの改善が見られなければ学習を停止
# restore_best_weights=True: 停止した際、最も検証ロスが良かった時点のモデルの重みに戻す
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# --- モデルの学習を開始 ---
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=100, # 多めに設定してもEarlyStoppingが止めてくれる
    batch_size=8, # データが少ないのでバッチサイズも小さく
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping],
    verbose=1 # 学習の進捗を表示
)

print("\n--- モデルの学習が完了しました！ ---")

In [ ]:
# --- Step 1: テストデータで予測を実行 ---
# model.predict() は、正規化されたスケール(0-1)で予測値を返します
predicted_scaled = model.predict(X_test_seq)


# --- Step 2: ★★★★★ 最重要 ★★★★★ 予測値を元のスケールに戻す ---
# 人間が理解できる「販売台数」のスケールに変換します
# このために、y専用のScaler (scaler_y) を使います
predicted_actual = scaler_y.inverse_transform(predicted_scaled)


# --- Step 3: 実際の正解データも元のスケールに戻して比較 ---
# y_test_scaled を使っても良いですが、分割前の y_test を使うのが簡単です
actual_data = y_test[-len(y_test_seq):] # シーケンス作成で削られた分を考慮
# もしくは、y_test_scaledを逆変換
# actual_data_from_scaled = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1))


print("\n--- 最終テスト結果 ---")
# X_test_seqは1つしかないので、予測結果も1つだけ出てきます
print(f"予測された来月の販売台数: {predicted_actual[0][0]:.2f} 台")
print(f"実際の来月の販売台数: {actual_data.iloc[0]['target_count_custom']} 台")

# 誤差を計算
error = predicted_actual[0][0] - actual_data.iloc[0]['target_count_custom']
print(f"予測誤差: {error:.2f} 台")

In [ ]:
# --- 1-a: 全ての正規化済みデータを結合する ---
X_full_scaled = np.concatenate([X_train_scaled, X_val_scaled, X_test_scaled], axis=0)
y_full_scaled = np.concatenate([y_train_scaled, y_val_scaled, y_test_scaled], axis=0)

print("--- 全データの形状 ---")
print(f"X_full_scaled shape: {X_full_scaled.shape}") # -> (37, 962)
print(f"y_full_scaled shape: {y_full_scaled.shape}") # -> (37, 1)

# --- 1-b: 結合した全データで、シーケンスを再作成 ---
lookback = 3 # 開発時と同じlookbackを使う
X_full_seq, y_full_seq = create_sequences(X_full_scaled, y_full_scaled, lookback)

print("\n--- 全データで作ったシーケンスの形状 ---")
print(f"X_full_seq shape: {X_full_seq.shape}") # -> (34, 3, 962)
print(f"y_full_seq shape: {y_full_seq.shape}") # -> (34,)


# --- 1-c: モデルを「再定義」し、「再学習」させる ---
# 以前と同じ設計図で、新しいモデルを組み立てる
model_final = Sequential([
    LSTM(units=64, return_sequences=True, input_shape=(X_full_seq.shape[1], X_full_seq.shape[2])),
    Dropout(0.2),
    LSTM(units=32, return_sequences=False),
    Dropout(0.2),
    Dense(units=16, activation='relu'),
    Dense(units=1)
])

model_final.compile(optimizer='adam', loss='mean_squared_error')

# 全データを使って学習 (今回は検証データはないので、validation_dataは不要)
print("\n--- 最終版モデルの学習開始！ ---")
model_final.fit(
    X_full_seq, y_full_seq,
    epochs=100, # EarlyStoppingがないので、ロスが下がらなくなるのを見計らって手動で止めても良い
    batch_size=8,
    verbose=1
)
print("\n--- 最終版モデルの学習完了！ ---")

In [ ]:
# --- 2-a: 予測の「元」となる最後の3ヶ月分のデータを準備 ---
# 元の月次集計データフレーム (正規化する前) から最後の3件を取得
last_sequence_raw = df_monthly.iloc[-lookback:]

# ★★★★★ 開発時に使った「scaler_X」で正規化する ★★★★★
# 新しくscalerをfitさせてはいけない！過去の基準で正規化するのが重要
last_sequence_scaled = scaler_X.transform(last_sequence_raw.drop('target_count_custom', axis=1))


# --- 2-b: LSTMの入力形式 (3次元) に変形 ---
# (1, タイムステップ数, 特徴量数) という形状にする
# 「1」は、これから1つだけ予測を行う、という意味
input_for_prediction = last_sequence_scaled.reshape(1, lookback, X_full_seq.shape[2])

print("\n--- 未来予測用の入力データ ---")
print(f"形状: {input_for_prediction.shape}")


# --- 2-c: 最終予測の実行！ ---
predicted_future_scaled = model_final.predict(input_for_prediction)

# --- 2-d: 予測値を元の「販売台数」スケールに戻す ---
# これも開発時に使った「scaler_y」を使う
predicted_future_actual = scaler_y.inverse_transform(predicted_future_scaled)


print("\n\n" + "="*40)
print("     未来予測の結果")
print("="*40)
print(f"AIが予測する【2025年2月】の販売台数: {predicted_future_actual[0][0]:.2f} 台")
print("="*40)

In [ ]:
# 最後の3ヶ月分の入力データ (前のステップで作成済み)
input_for_prediction = last_sequence_scaled.reshape(1, lookback, X_full_seq.shape[2])

# 何ヶ月先まで予測したいか
n_future_steps = 3

# 予測結果を格納するリスト
future_predictions = []

# 現在の入力を保持する変数
current_input = input_for_prediction.copy()

print("--- 再帰的予測を開始 ---")
for i in range(n_future_steps):
    # 1. 次の1ヶ月を予測
    next_step_pred_scaled = model_final.predict(current_input)

    # 2. 予測結果をリストに保存
    future_predictions.append(next_step_pred_scaled[0])

    # 3. 次の予測のための新しい入力を作成
    #    - 入力の最後の特徴量ベクトルに、今回の予測値を差し込む (ここは簡易的な方法)
    #    - 注：実際には他の961個の特徴量は不明なので、前の月の値を使うなど仮定が必要。
    #      ここではターゲット(y)だけを更新する簡易版を示します。
    #      より厳密には、特徴量も予測するモデルが必要になります。

    # 簡易的な次の入力の作成 (ターゲット予測値だけを更新する例)
    # 実際にはもっと複雑な処理が必要になる場合が多い
    new_features_for_next_step = current_input[0, -1, :].copy() # 最後の月の特徴量
    # 本来はこの部分でy以外の特徴量も予測・更新する必要がある

    # ここでは簡易化のため、入力シーケンスを1つずらし、
    # 新しい予測値を（仮の特徴量ベクトルとして）追加するイメージで進めます。
    # 注：この方法はあくまでコンセプトを示すもので、特徴量の扱いには注意が必要です。

    # よりシンプルな方法は、入力シーケンスを「ずらす」ことです
    # 最初のタイムステップを削除
    next_input_features = current_input[0, 1:, :]

    # 予測結果を新しいタイムステップとして追加
    # 予測結果はyのみなので、xの形状に合わせる必要がある（ここではダミー値で代用）
    predicted_y = next_step_pred_scaled[0][0]
    dummy_x_features = np.zeros(current_input.shape[2] - 1) # y以外の特徴量を0で埋めるダミー
    new_timestep = np.insert(dummy_x_features, 0, predicted_y).reshape(1, current_input.shape[2])

    current_input = np.append(next_input_features, new_timestep, axis=0).reshape(1, lookback, X_full_seq.shape[2])


# 予測結果を元のスケールに戻す
future_predictions_actual = scaler_y.inverse_transform(np.array(future_predictions))

print("\n--- 複数ステップ先の予測結果 ---")
for i, pred in enumerate(future_predictions_actual):
    print(f"【{i+1}ヶ月先】の予測販売台数: {pred[0]:.2f} 台")

In [ ]:
n_future_steps = 12 # 3ヶ月先まで予測する
future_predictions = []

# 履歴を保持するためのデータフレーム（これに予測結果を追記していく）
df_history = df_monthly.copy()

# 予測ループの開始
for i in range(n_future_steps):

    # --- 1-a: 次の月の特徴量Xを作成する ---
    last_month = df_history.iloc[-1]
    # 12ヶ月前（前年同月）と13ヶ月前のデータを取得
    last_year_same_month = df_history.iloc[-12]

    # ★★★★★ あなたの戦略Aを実装 ★★★★★
    future_X_values = (last_year_same_month + last_month) / 2

    # ターゲット列はまだ不明なので、一旦仮の値（0など）を入れておく
    future_X_df = pd.DataFrame(future_X_values).T
    future_X_df['target_count_custom'] = 0 # ターゲット列はXに含まない
    future_X_for_scale = future_X_df.drop('target_count_custom', axis=1)


    # --- 1-b: 予測の入力シーケンスを作成 ---
    # 履歴の最後から (lookback-1) 件を取得
    last_sequence_raw = df_history.iloc[-(lookback-1):].drop('target_count_custom', axis=1)
    # 1-aで作成した未来の特徴量と結合
    input_sequence_raw = pd.concat([last_sequence_raw, future_X_for_scale])


    # --- 1-c: 正規化と形状変換 ---
    input_sequence_scaled = scaler_X.transform(input_sequence_raw)
    input_for_prediction = input_sequence_scaled.reshape(1, lookback, df_monthly.shape[1] - 1)


    # --- 1-d: 予測の実行と保存 ---
    predicted_scaled = model_final.predict(input_for_prediction)
    predicted_actual = scaler_y.inverse_transform(predicted_scaled)
    future_predictions.append(predicted_actual[0][0])


    # --- 1-e: 履歴の更新 ★★★★★
    # 次のループのために、今予測した値を履歴に追加する
    new_row = future_X_df.copy()
    new_row.index = [df_history.index[-1] + pd.DateOffset(months=1)]
    new_row['target_count_custom'] = predicted_actual[0][0]

    df_history = pd.concat([df_history, new_row])


print("\n" + "="*40)
print(f"     未来{n_future_steps}ヶ月の再帰的予測結果")
print("="*40)
for i, pred in enumerate(future_predictions):
    future_month = df_monthly.index[-1] + pd.DateOffset(months=i+1)
    print(f"【{future_month.strftime('%Y-%m')}】の予測販売台数: {pred:.2f} 台")
print("="*40)

In [ ]:
import matplotlib.font_manager as fm

font_path = '/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc' # for Mac
# font_path = 'C:/Windows/Fonts/YuGothM.ttc' # for Windows
try:
    font_prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = font_prop.get_name()
    print(f"日本語フォント '{font_prop.get_name()}' を設定しました。")
except FileNotFoundError:
    print(f"警告: 指定されたフォントパス '{font_path}' が見つかりません。デフォルトフォントで描画します。")

dates = pd.date_range(start='2022-01-31', periods=37, freq='M')


# --- Step 1: 予測データに対応する未来の日付を作成 ---
last_historical_date = df_monthly.index[-1]
future_dates = pd.date_range(start=last_historical_date, periods=len(future_predictions) + 1, freq='M')[1:]

# --- Step 2: 予測値をPandas Seriesに変換 ---
future_predictions_series = pd.Series(future_predictions, index=future_dates)


# --- Step 3: グラフの描画 ---
plt.style.use('seaborn-v0_8-whitegrid') # グラフのスタイルを指定
plt.figure(figsize=(16, 8)) # グラフのサイズを大きく見やすく設定

# 3-a: 実績値のプロット
plt.plot(
    df_monthly.index,
    df_monthly['target_count_custom'],
    label='実績値 (Historical Actuals)',
    color='royalblue',
    marker='o',
    linewidth=2
)

# 3-b: 予測値のプロット
plt.plot(
    future_predictions_series.index,
    future_predictions_series.values,
    label='予測値 (Future Predictions)',
    color='darkorange',
    marker='x',
    linestyle='--',
    linewidth=2,
    markersize=8
)

# 3-c: 実績と予測の境界線に縦線を引く
plt.axvline(x=last_historical_date, color='gray', linestyle=':', linewidth=2, label='予測開始時点')


# --- Step 4: グラフの装飾 ---
plt.title('monthly_AI_prediction', fontsize=20, pad=20)
plt.xlabel('year_date', fontsize=14)
plt.ylabel('contract_sample', fontsize=14)
plt.xticks(rotation=45) # 日付ラベルを斜めにして見やすくする
plt.legend(fontsize=12) # 凡例を表示
plt.grid(True)
plt.tight_layout() # レイアウトを自動調整

plt.savefig("prediction.png", dpi=300, bbox_inches='tight')

# グラフを表示
plt.show()